In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("spark://10.99.250.55:7077").setAppName("Books")
conf.set("spark.executor.memory", "6G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "8") # number of cores in each executor
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "16") # total number of parallelism across all nodes
spark_context =  SparkContext.getOrCreate(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/22 23:05:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
books_file = spark_context.textFile("./BX-CSV-Dump/BX-Books.csv")
print("number of books = ",books_file.count())
print("First 3 rows are - \n",books_file.take(3))

number of books =  271380
First 3 rows are - 
 ['"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"', '"0195153448";"Classical Mythology";"Mark P. O. Morford";"2002";"Oxford University Press";"http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg"', '"0002005018";"Clara Callan";"Richard Bruce Wright";"2001";"HarperFlamingo Canada";"http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg"']


In [4]:
books_file = books_file.map(lambda l: l.split(';'))
print("number of books = ",books_file.count())
print("First 3 rows are - \n",books_file.take(3))

number of books =  271380
First 3 rows are - 
 [['"ISBN"', '"Book-Title"', '"Book-Author"', '"Year-Of-Publication"', '"Publisher"', '"Image-URL-S"', '"Image-URL-M"', '"Image-URL-L"'], ['"0195153448"', '"Classical Mythology"', '"Mark P. O. Morford"', '"2002"', '"Oxford University Press"', '"http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg"', '"http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg"', '"http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg"'], ['"0002005018"', '"Clara Callan"', '"Richard Bruce Wright"', '"2001"', '"HarperFlamingo Canada"', '"http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg"', '"http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg"', '"http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg"']]


In [5]:
#import the BX-Book-Ratings.csv file and split it into rows with individual elements
ratings_file = spark_context.textFile("./BX-CSV-Dump/BX-Book-Ratings.csv").map(lambda l: l.split('";"'))
print("First 3 rows are - \n",ratings_file.take(10))
print(" \n K,V pairs are - \n", ratings_file.map(lambda x: (x[1],1)).take(10))

First 3 rows are - 
 [['"User-ID', 'ISBN', 'Book-Rating"'], ['"276725', '034545104X', '0"'], ['"276726', '0155061224', '5"'], ['"276727', '0446520802', '0"'], ['"276729', '052165615X', '3"'], ['"276729', '0521795028', '6"'], ['"276733', '2080674722', '0"'], ['"276736', '3257224281', '8"'], ['"276737', '0600570967', '6"'], ['"276744', '038550120X', '7"']]
 
 K,V pairs are - 
 [('ISBN', 1), ('034545104X', 1), ('0155061224', 1), ('0446520802', 1), ('052165615X', 1), ('0521795028', 1), ('2080674722', 1), ('3257224281', 1), ('0600570967', 1), ('038550120X', 1)]


In [6]:
ratings_kv = ratings_file.filter(lambda x: x[1] != 'ISBN' ).map(lambda x: (x[1],1))
#print(ratings_kv.take(10))
ratings_count = ratings_kv.reduceByKey(lambda x, y: x + y)
print(ratings_count.take(10))

[('0671038567', 57), ('0671521462', 1), ('0671708635', 143), ('0679405283', 62), ('0743237188', 284), ('0812524330', 8), ('0842329250', 107), ('3442431875', 18), ('0156006294', 5), ('0312112947', 2)]


In [7]:
ratings_sorted = ratings_count.map(lambda x: (x[1], x[0])).sortByKey(False)

In [8]:
top_10 = []
for i in ratings_sorted.take(10):
    top_10.append('"'+i[1]+'"')
print(books_file.filter(lambda x: x[0] in top_10).map(lambda x: x[1]).collect())


['"Wild Animus"', '"Angels &amp', '"A Painted House"', '"The Secret Life of Bees"', '"The Lovely Bones: A Novel"', '"The Red Tent (Bestselling Backlist)"', '"The Da Vinci Code"', '"Divine Secrets of the Ya-Ya Sisterhood: A Novel"', '"Snow Falling on Cedars"']
